In [3]:
import cv2
import numpy as np

In [4]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = int(image.shape[0])
    y2 = int(y1*3/5)
    x1 = int((y1 - intercept)/slope) # y = m*x + b
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return None
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis = 0)
    right_fit_average = np.average(right_fit, axis = 0)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array((left_line, right_line))


def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # let the machine work with the grayscale image, as gray is ranging from 0 to 255 in onnly 1 channel instead of 3 (RGB) less computational power needed 
    blur = cv2.GaussianBlur(gray, (5, 5), 0) # Noise reduction using Canny/Blur (Noises can create false Edge detection)
    canny = cv2.Canny(blur, 50, 150) # Documentation recommends to use a ratio of 1 : 3 to the low_Threshold and high_threshold. we take then 50 : 150
    return canny

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10) # we draw 2 blue lines with thikness of 10 along the lanes on the image
    return line_image

def region_of_interest(image): # enclose the region of our view, in order to let the machine concentrate only on the lanes (drawing a Triangle like pattern)
    
    height = image.shape[0]
    polygons = np.array([
    [((200, height), (1100, height), (550, 250))]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

cap = cv2.VideoCapture('test2.mp4')
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image) # the gradient image with major changes of Color Intensity
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1) # combination of our image and blue generated lanes
        
    cv2.imshow('result', combo_image) # display the image
    if cv2.waitKey(1) & 0xFF == ord('q'): # 1 ms wait between each frame of the video. if O then the video will freeze
        break
        
cap.release()
cv2.destroyAllWindoes()